# LIBARY

In [146]:
import geopandas as gpd
import pandas as pd
import numpy as np
import fiona
import pyogrio
import os

import requests
import zipfile
import os
from io import BytesIO
from bs4 import BeautifulSoup

# READ EVENT DATA

In [147]:
def read_event_data(eventid = 'nc72282711'):
    """
    Read event data from a GPKG file.
    """
    parent_dir = os.path.dirname(os.getcwd())
    event_dir = os.path.join(parent_dir, 'ShakeMaps', eventid)

    # Update with the actual path
    GPKG_PATH = os.path.join(event_dir, "eqmodel_outputs.gpkg")

    # Read the layer you want to inspect
    # tract_shakemap_mmi, tract_shakemap_pga, tract_shakemap_pgv --> same idea
    gdf = gpd.read_file(GPKG_PATH, layer="tract_shakemap_pga")
    # make sure that only row that is not nan is the one we want
    columns = gdf.columns
    gdf = gdf[[columns[0], columns[1], columns[2], columns[3], columns[-1]]]
    
    return gdf.loc[gdf[columns[1]].notna()]

In [148]:
eventdata = read_event_data()
eventdata.columns

Index(['GEOID', 'max_intensity', 'min_intensity', 'mean_intensity',
       'geometry'],
      dtype='object')

# READ BUILDING DATA

In [149]:
# In this module, assume that csv files are already exist in the directory.

In [150]:
# Check if a csv file for a state is exists
    # if exists, read it
    # if not, check if the gdb file exists
    # if exists, read it
def read_building_count_by_tract(stateid):
    """
    Read building count data from a CSV file.
    """
    parent_dir = os.path.dirname(os.getcwd())
    # Update with the actual path
    CSV_PATH = os.path.join(parent_dir, 'Data', 'building_data_csv', f"{stateid}_building_data.csv")
    # check if the file exists
    if not os.path.exists(CSV_PATH):
        print(f"CSV file for Building count data {stateid} is not available.")
        return None
    else:
        gdf = pd.read_csv(CSV_PATH, dtype={'CENSUSCODE': str})
        return gdf

In [151]:
states_data = [
    ("Alabama", "AL"), ("Alaska", "AK"), ("Arizona", "AZ"), ("Arkansas", "AR"),
    ("California", "CA"), ("Colorado", "CO"), ("Connecticut", "CT"), ("Delaware", "DE"),
    ("Florida", "FL"), ("Georgia", "GA"), ("Hawaii", "HI"), ("Idaho", "ID"),
    ("Illinois", "IL"), ("Indiana", "IN"), ("Iowa", "IA"), ("Kansas", "KS"),
    ("Kentucky", "KY"), ("Louisiana", "LA"), ("Maine", "ME"), ("Maryland", "MD"),
    ("Massachusetts", "MA"), ("Michigan", "MI"), ("Minnesota", "MN"), ("Mississippi", "MS"),
    ("Missouri", "MO"), ("Montana", "MT"), ("Nebraska", "NE"), ("Nevada", "NV"),
    ("New Hampshire", "NH"), ("New Jersey", "NJ"), ("New Mexico", "NM"), ("New York", "NY"),
    ("North Carolina", "NC"), ("North Dakota", "ND"), ("Ohio", "OH"), ("Oklahoma", "OK"),
    ("Oregon", "OR"), ("Pennsylvania", "PA"), ("Rhode Island", "RI"), ("South Carolina", "SC"),
    ("South Dakota", "SD"), ("Tennessee", "TN"), ("Texas", "TX"), ("Utah", "UT"),
    ("Vermont", "VT"), ("Virginia", "VA"), ("Washington", "WA"), ("West Virginia", "WV"),
    ("Wisconsin", "WI"), ("Wyoming", "WY")
]

In [152]:
read_building_count_by_tract("CA").dtypes

CENSUSCODE                   object
OTHER_OTHER                   int64
RESIDENTIAL_MULTI FAMILY      int64
RESIDENTIAL_OTHER             int64
RESIDENTIAL_SINGLE FAMILY     int64
TOTAL_RESIDENTIAL             int64
TOTAL_BUILDING                int64
dtype: object

# INTERSECT WITH BUILDING STOCKS

In [153]:
def get_building_stock_data():
    """
    2. Check if the csv file exists
    3. If not, create the folder aand copy the csv file
    4. If exists, read the csv file
    """

    parent_dir = os.path.dirname(os.getcwd())
    # check if the folder exists
    CSV_PATH = os.path.join(parent_dir, 'Data', 'building_stock_data', 'Building_Percentages_Per_Tract_ALLSTATES.csv')
    

    if os.path.exists(CSV_PATH):
        print(f"Building stock data exists at {CSV_PATH}")
        gdf = gpd.read_file(CSV_PATH)
        gdf['CENSUSCODE'] = np.where(gdf['Tract'].str.len() == 11, gdf['Tract'], "0"+gdf['Tract'])

    else:
        print(f"Building stock data does not exist at {CSV_PATH}")
        # create or download the files
        pass
    
    return gdf

In [154]:
building_stock = get_building_stock_data()
building_stock.dtypes

Building stock data exists at /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/building_stock_data/Building_Percentages_Per_Tract_ALLSTATES.csv


field_1       object
Tract         object
W1            object
W2            object
S1L           object
S1M           object
S1H           object
S2L           object
S2M           object
S2H           object
S3            object
S4L           object
S4M           object
S4H           object
S5L           object
S5M           object
S5H           object
C1L           object
C1M           object
C1H           object
C2L           object
C2M           object
C2H           object
C3L           object
C3M           object
C3H           object
PC1           object
PC2L          object
PC2M          object
PC2H          object
RM1L          object
RM1M          object
RM2L          object
RM2M          object
RM2H          object
URML          object
URMM          object
MH            object
Total         object
CENSUSCODE    object
dtype: object

# JOIN COUNT BUILDING DATA AND BUILDING STOCK DATA

In [155]:
# take df_pivot and building_stock and merge them
def count_building_proportion(building_count, building_stock):
    # merge the dataframes
    merged_df = pd.merge(building_count, building_stock, on='CENSUSCODE', how='left')
    merged_df.drop(columns=['Tract'], axis=1, inplace=True)
    merged_df.drop(columns=['field_1'], axis=1, inplace=True)
    merged_df.ffill(inplace=True)
    return merged_df

In [156]:

df_output = count_building_proportion(read_building_count_by_tract("CA"), get_building_stock_data())
df_output

Building stock data exists at /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/building_stock_data/Building_Percentages_Per_Tract_ALLSTATES.csv


,CENSUSCODE,OTHER_OTHER,RESIDENTIAL_MULTI FAMILY,RESIDENTIAL_OTHER,RESIDENTIAL_SINGLE FAMILY,TOTAL_RESIDENTIAL,TOTAL_BUILDING,W1,W2,S1L,...,PC2H,RM1L,RM1M,RM2L,RM2M,RM2H,URML,URMM,MH,Total
0,06001400100,104,22,10,1162,1194,1298,0.9312280701754386,0.011228070175438596,0.004912280701754386,...,0.0,0.021052631578947368,0.0,0.0014035087719298245,0.0,0.0,0.0035087719298245615,0.0,0.0,1.0007017543859649
1,06001400200,46,110,2,538,650,696,0.8804483188044832,0.0186799501867995,0.009962640099626401,...,0.0,0.0323785803237858,0.0,0.0024906600249066002,0.0,0.0,0.0062266500622665,0.0,0.0,1.0024906600249066
2,06001400300,67,416,7,1139,1562,1629,0.8618225134008338,0.02084574151280524,0.009529481834425254,...,0.0,0.03692674210839786,0.0,0.0023823704586063135,0.0,0.0,0.008933889219773675,0.0,0.0,1.0000000000000002
3,06001400400,57,391,3,777,1171,1228,0.8871315600287563,0.015815959741193385,0.005751258087706686,...,0.0,0.03163191948238677,0.0,0.0014378145219266715,0.0,0.0,0.007189072609633357,0.0,0.0,0.9985621854780734
4,06001400500,56,342,6,614,962,1018,0.8730305838739574,0.014828544949026877,0.005560704355885079,...,0.0,0.03521779425393883,0.0,0.0018535681186283596,0.0,0.0,0.009267840593141797,0.0,0.0,1.0009267840593143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9108,06115040902,759,0,0,0,0,759,0.8811013767209012,0.011264080100125156,0.007509386733416771,...,0.0,0.03128911138923655,0.0,0.0025031289111389237,0.0,0.0,0.007509386733416771,0.0,0.0050062578222778474,0.997496871088861
9109,06115041001,579,90,360,1807,2257,2836,0.8811013767209012,0.011264080100125156,0.007509386733416771,...,0.0,0.03128911138923655,0.0,0.0025031289111389237,0.0,0.0,0.007509386733416771,0.0,0.0050062578222778474,0.997496871088861
9110,06115041002,270,155,628,1771,2554,2824,0.8811013767209012,0.011264080100125156,0.007509386733416771,...,0.0,0.03128911138923655,0.0,0.0025031289111389237,0.0,0.0,0.007509386733416771,0.0,0.0050062578222778474,0.997496871088861
9111,06115041101,373,86,596,903,1585,1958,0.8811013767209012,0.011264080100125156,0.007509386733416771,...,0.0,0.03128911138923655,0.0,0.0025031289111389237,0.0,0.0,0.007509386733416771,0.0,0.0050062578222778474,0.997496871088861


# JOIN WITH EVENT DATA


In [157]:
final_output = pd.merge(eventdata, df_output, left_on='GEOID', right_on='CENSUSCODE', how='left')
final_output.ffill(inplace=True)
final_output.drop(columns=['CENSUSCODE'], axis=1, inplace=True)
final_output.columns

Index(['GEOID', 'max_intensity', 'min_intensity', 'mean_intensity', 'geometry',
       'OTHER_OTHER', 'RESIDENTIAL_MULTI FAMILY', 'RESIDENTIAL_OTHER',
       'RESIDENTIAL_SINGLE FAMILY', 'TOTAL_RESIDENTIAL', 'TOTAL_BUILDING',
       'W1', 'W2', 'S1L', 'S1M', 'S1H', 'S2L', 'S2M', 'S2H', 'S3', 'S4L',
       'S4M', 'S4H', 'S5L', 'S5M', 'S5H', 'C1L', 'C1M', 'C1H', 'C2L', 'C2M',
       'C2H', 'C3L', 'C3M', 'C3H', 'PC1', 'PC2L', 'PC2M', 'PC2H', 'RM1L',
       'RM1M', 'RM2L', 'RM2M', 'RM2H', 'URML', 'URMM', 'MH', 'Total'],
      dtype='object')

# SAVE OUTPUT TO EVENT DIR


In [158]:
# Function to save GeoDataFrame to GeoPackage (Overwriting mode)
def save_to_geopackage(gdf, layer_name="tract_shakemap_pga", eventid = 'nc72282711'):
    """
    Saves a GeoDataFrame to the GeoPackage, overwriting the existing layer.

    Args:
        gdf (GeoDataFrame): The GeoDataFrame to save.
        layer_name (str): The name of the layer in the GeoPackage.
    """
    parent_dir = os.path.dirname(os.getcwd())
    event_dir = os.path.join(parent_dir, 'ShakeMaps', eventid)

    # Update with the actual path
    GPKG_PATH = os.path.join(event_dir, "eqmodel_outputs.gpkg")


    gdf.to_file(GPKG_PATH, layer=layer_name, driver="GPKG", mode="w")
    print(f"Saved {layer_name} to {GPKG_PATH} (overwritten).")

save_to_geopackage(final_output, layer_name="tract_shakemap_pga", eventid = 'nc72282711')

Saved tract_shakemap_pga to /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc72282711/eqmodel_outputs.gpkg (overwritten).


In [159]:
parent_dir = os.path.dirname(os.getcwd())
event_dir = os.path.join(parent_dir, 'ShakeMaps', 'nc72282711')

# Update with the actual path
GPKG_PATH = os.path.join(event_dir, "eqmodel_outputs.gpkg")
gpd.read_file(GPKG_PATH, layer="tract_shakemap_pga")

,GEOID,max_intensity,min_intensity,mean_intensity,OTHER_OTHER,RESIDENTIAL_MULTI FAMILY,RESIDENTIAL_OTHER,RESIDENTIAL_SINGLE FAMILY,TOTAL_RESIDENTIAL,TOTAL_BUILDING,...,RM1L,RM1M,RM2L,RM2M,RM2H,URML,URMM,MH,Total,geometry
0,06001400100,0.02,0.02,0.02,104.0,22.0,10.0,1162.0,1194.0,1298.0,...,0.021052631578947368,0.0,0.0014035087719298245,0.0,0.0,0.0035087719298245615,0.0,0.0,1.0007017543859649,"MULTIPOLYGON (((-122.24692 37.88544, -122.2466..."
1,06001400200,0.02,0.02,0.02,46.0,110.0,2.0,538.0,650.0,696.0,...,0.0323785803237858,0.0,0.0024906600249066002,0.0,0.0,0.0062266500622665,0.0,0.0,1.0024906600249066,"MULTIPOLYGON (((-122.25792 37.84261, -122.2577..."
2,06001400300,0.02,0.02,0.02,67.0,416.0,7.0,1139.0,1562.0,1629.0,...,0.03692674210839786,0.0,0.0023823704586063135,0.0,0.0,0.008933889219773675,0.0,0.0,1.0000000000000002,"MULTIPOLYGON (((-122.26563 37.83764, -122.2655..."
3,06001400400,0.02,0.02,0.02,57.0,391.0,3.0,777.0,1171.0,1228.0,...,0.03163191948238677,0.0,0.0014378145219266715,0.0,0.0,0.007189072609633357,0.0,0.0,0.9985621854780734,"MULTIPOLYGON (((-122.26183 37.84162, -122.2618..."
4,06001400500,0.02,0.02,0.02,56.0,342.0,6.0,614.0,962.0,1018.0,...,0.03521779425393883,0.0,0.0018535681186283596,0.0,0.0,0.009267840593141797,0.0,0.0,1.0009267840593143,"MULTIPOLYGON (((-122.26951 37.84858, -122.2693..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,06115040800,0.01,0.01,0.01,454.0,128.0,136.0,1256.0,1520.0,1974.0,...,0.024025974025974027,0.0,0.001948051948051948,0.0,0.0,0.005844155844155844,0.0,0.04935064935064935,1.0006493506493503,"MULTIPOLYGON (((-121.51553 39.03064, -121.5153..."
2601,06115040901,0.01,0.01,0.01,388.0,109.0,491.0,1025.0,1625.0,2013.0,...,0.016681299385425813,0.0,0.001755926251097454,0.0,0.0,0.003511852502194908,0.0,0.3520632133450395,1.0008779631255487,"MULTIPOLYGON (((-121.58338 39.13621, -121.5831..."
2602,06115040902,0.01,0.01,0.01,759.0,0.0,0.0,0.0,0.0,759.0,...,0.03128911138923655,0.0,0.0025031289111389237,0.0,0.0,0.007509386733416771,0.0,0.0050062578222778474,0.997496871088861,"MULTIPOLYGON (((-121.47722 39.13334, -121.4770..."
2603,06115041000,0.01,0.01,0.01,759.0,0.0,0.0,0.0,0.0,759.0,...,0.03128911138923655,0.0,0.0025031289111389237,0.0,0.0,0.007509386733416771,0.0,0.0050062578222778474,0.997496871088861,"MULTIPOLYGON (((-121.63637 39.24608, -121.6362..."
